In [8]:
import string

import torch
import nltk
nltk.download()
from nltk import word_tokenize, MWETokenizer
from nltk.corpus import stopwords
from transformers import BertTokenizer, BertForMaskedLM

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [7]:
nltk.downlaod()

AttributeError: module 'nltk' has no attribute 'downlaod'

In [13]:
def mask_and_predict(sentence, top_n=3):
    # Load pre-trained model and tokenizer
    model_name = 'bert-base-uncased'
    model = BertForMaskedLM.from_pretrained(model_name)
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model.eval()

    words = sentence.split()
    results = []

    # Loop through each word in the sentence
    for i in range(len(words)):
        masked_sentence = ' '.join([words[j] if j != i else '[MASK]' for j in range(len(words))])
        inputs = tokenizer(masked_sentence, return_tensors="pt")

        # Get model output
        with torch.no_grad():
            output = model(**inputs)
            logits = output.logits

        # Get top_n predictions for the masked word
        masked_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)[0].item()
        probs = logits[0, masked_index].softmax(dim=0)
        top_prob_values, top_indices = torch.topk(probs, top_n)

        print(f"Original Word: {words[i]}")
        for idx, token_id in enumerate(top_indices):
            predicted_token = tokenizer.convert_ids_to_tokens([token_id.item()])[0]
            prob_value = top_prob_values[idx].item()
            print(f"   Candidate {idx + 1}: {predicted_token} with probability {prob_value:.4f}")

        top_prediction = tokenizer.convert_ids_to_tokens([top_indices[0].item()])[0]
        results.append((words[i], top_prediction, top_prob_values[0].item()))

    return results

In [14]:
def remove_punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))

In [16]:
sentence = """If you burn the candle on both ends, you’re not as bright as you think."""
tokenizer = MWETokenizer()  # Multi-word expression tokenizer
tokens = tokenizer.tokenize(word_tokenize(sentence))
stop_words = set(stopwords.words('english'))
# filtered_tokens = [w for w in tokens if not w.lower() in stop_words]
# print(" ".join(filtered_tokens))
predictions = mask_and_predict(" ".join(tokens))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Original Word: If
   Candidate 1: if with probability 0.6204
   Candidate 2: when with probability 0.2666
   Candidate 3: unless with probability 0.0627
Original Word: you
   Candidate 1: you with probability 0.9910
   Candidate 2: i with probability 0.0039
   Candidate 3: they with probability 0.0020
Original Word: burn
   Candidate 1: light with probability 0.1434
   Candidate 2: hold with probability 0.1374
   Candidate 3: see with probability 0.0661
Original Word: the
   Candidate 1: a with probability 0.7958
   Candidate 2: the with probability 0.1513
   Candidate 3: your with probability 0.0157
Original Word: candle
   Candidate 1: fuse with probability 0.0471
   Candidate 2: wires with probability 0.0209
   Candidate 3: skin with probability 0.0163
Original Word: on
   Candidate 1: on with probability 0.4693
   Candidate 2: at with probability 0.4605
   Candidate 3: from with probability 0.0213
Original Word: both
   Candidate 1: both with probability 0.6140
   Candidate 2: the 